In [2]:
import tensorflow as tf
import numpy as np
import gradio as gr
from skimage.transform import resize
from keras import utils

In [3]:
# Load saved model from model.tflite file
interpreter = tf.lite.Interpreter(model_path="../model.tflite")
interpreter.allocate_tensors()

print(interpreter.get_input_details())
print(interpreter.get_output_details())

[{'name': 'serving_default_conv2d_6_input:0', 'index': 0, 'shape': array([  1, 224, 224,   3]), 'shape_signature': array([ -1, 224, 224,   3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 25, 'shape': array([1, 4]), 'shape_signature': array([-1,  4]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [4]:
# Classifier
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

def classify_img(img):
    # preprocessing: resizing to appropriate shape, making pixel values between 0 and 1
    resized = resize(img, (224, 224))
    new_arr = resized.astype(np.float32)

    interpreter.set_tensor(input_details[0]['index'], [new_arr])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    results = {}
    results["Heavy damage"] = output_data[0][0]
    results["Minor damage"] = output_data[0][1]
    results["Moderate damage"] = output_data[0][2]
    results["Undamaged"] = output_data[0][3]
    return results

In [4]:
# Create demo with Gradio
demo = gr.Interface(fn=classify_img, inputs=gr.Image(type="numpy"), outputs="label")
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [43]:
# Finding accuracy per category

# Load dataset
test_images = np.load("../dataset/task7_X_test.npy")
test_labels = np.load("../dataset/task7_y_test.npy")

# Normalize
def normalize(images):
    return (images + 123.68)/274.75
test_images = normalize(test_images)

# Reformat the labels to be 0-3 and categorical instead of [1, 0, 0, 0]-[0, 0, 0, 1]
def re_format(old_labels):
    labels = np.zeros((old_labels.shape[0], 1))
    for i in range(len(old_labels)):
        for j in range(len(old_labels[i])):
            if old_labels[i][j] == 1:
                labels[i] = j
                break
    return labels
test_labels_f = re_format(test_labels)
test_labels = utils.to_categorical(test_labels_f, 4)

# predict each test image
correct = [0, 0, 0, 0]
total = [0, 0, 0, 0]
for i in range(0,len(test_images)):
    # format labels
    label_names = ["Heavy damage", "Minor damage", "Moderate damage", "Undamaged"]
    label_index = 0
    for j in range(len(test_labels[i])):
        if test_labels[i][j] == 1:
            label_index = j
            break
    label = label_names[label_index]

    # predict
    score = classify_img(test_images[i])

    # find best prediction
    max = 0
    lmax = ""
    for l in label_names:
        if score[l] > max:
            max = score[l]
            lmax = l

    # record if correct based on label
    if lmax == label:
        correct[label_index] += 1
        total[label_index] += 1
    else:
        total[label_index] += 1

# report accuracies 
for i in range(0,4):
    print("{} had a {:.2f} percent accuracy."
      .format(label_names[i], (correct[i] / total[i]) * 100))


Heavy damage had a 76.60 percent accuracy.
Minor damage had a 55.91 percent accuracy.
Moderate damage had a 38.46 percent accuracy.
Undamaged had a 71.01 percent accuracy.
